# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
number = '19'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Open clean dataset

In [3]:
data_file = pd.read_csv(analyze_dir + 'csv_files/clean_files/clean_files_forum_' + number + '.csv')
data_file

file_number        message_id
0            660000  message #1257319
1            660001  message #1257320
2            660002  message #1257321
3            660003  message #1257323
4            660004  message #1257324
5            660005  message #1257325
6            660006  message #1257326
7            660007  message #1257327
8            660008  message #1257328
9            660009  message #1257329
10           660010  message #1257330
11           660011  message #1257331
12           660012  message #1257332
13           660013  message #1257333
14           660014  message #1257334
15           660015  message #1257335
16           660016  message #1257336
17           660017  message #1257337
18           660018  message #1257339
19           660019  message #1257340
20           660020  message #1257341
21           660021  message #1257342
22           660022  message #1257343
23           660023  message #1257344
24           660024  message #1257346
25           660025  message #1257347
26           660026  message #1257348
27           660027  message #1257349
28           660028  message #1257350
29           660029  message #1257351
...             ...               ...
189368       879962  message #1497768
189369       879963  message #1497769
189370       879965  message #1497771
189371       879966  message #1497772
189372       879967  message #1497773
189373       879968  message #1497774
189374       879970  message #1497776
189375       879971  message #1497777
189376       879972  message #1497778
189377       879973  message #1497779
189378       879975  message #1497781
189379       879976  message #1497782
189380       879977  message #1497783
189381       879978  message #1497784
189382       879980  message #1497786
189383       879981  message #1497787
189384       879982  message #1497788
189385       879983  message #1497789
189386       879985  message #1497791
189387       879986  message #1497792
189388       879987  message #1497793
189389       879988  message #1497794
189390       879990  message #1497796
189391       879991  message #1497797
189392       879992  message #1497798
189393       879993  message #1497799
189394       879995  message #1497801
189395       879996  message #1497802
189396       879997  message #1497803
189397       879998  message #1497804

[189398 rows x 2 columns]

In [4]:
data_file.iloc[1][0]

660001

# Get links from files

In [5]:
print ("Forum files: %s" %(number))
url_data = []
for file_num in range(0,len(data_file)):
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(data_file)))
    sys.stdout.flush()

    file = dataset + str(data_file.iloc[file_num][0])
    f = open(file)
    f = f.read()

    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [data_file.iloc[file_num][0], msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a'):
                    username = usr.text
                    if username == '':
                        username = 'Eclipse User'

                    for link in userlink.find_all('span', attrs={'class':'MsgBodyText'}):
                        for atag in link.find_all('a'):
                            ss = atag.text.replace(' ','')
                            parser = []
                            try:
                                link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                                link = link.group()
                                domain = urlparse(link)
                                tld_dt = get_tld(link)
                                parser = [username, link, domain.netloc, tld_dt]
                                data.insert(len(data),parser)
                            except:
                                pass

        n = 3
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][1] not in link_name:
                url_data.append([data[0], data[1], data[2], data[n][0], data[n][1], data[n][2], data[n][3]])
                link_name.append(data[n][1])
            else:
                pass
            n = n + 1
    
    except:
        pass

print ('\nFiles extraction are finished')

Forum files: 19
Extracting file number: 189398 / 189398
Files extraction are finished


In [6]:
url_data

[[660000,
  'message #1257319',
  'Welcome to eclipse.cbi',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [660000,
  'message #1257319',
  'Welcome to eclipse.cbi',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.cbi',
  'www.eclipse.org',
  'org'],
 [660001,
  'message #1257320',
  'Welcome to eclipse.mbeddr',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [660001,
  'message #1257320',
  'Welcome to eclipse.mbeddr',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.mbeddr',
  'www.eclipse.org',
  'org'],
 [660003,
  'message #1257323',
  'Welcome to eclipse.kura',
  'Eclipse Webmaster',
  'http://www.eclipse.org/proposals/',
  'www.eclipse.org',
  'org'],
 [660003,
  'message #1257323',
  'Welcome to eclipse.kura',
  'Eclipse Webmaster',
  'http://www.eclipse.org/forums/eclipse.kura',
  'www.eclipse.org',
  'org'],
 [660004,
  'message #1257324',
  'Welcome to 

In [7]:
with open (analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'topic', 'author', 'link', 'domain', 'top_level_domain']
    writers.writerow(header)
    for item in url_data:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_link = pd.read_csv(analyze_dir + 'csv_files/links_list/link_list_forumfiles_' + number + '.csv')
df_link

file_number        message_id                      topic  \
0            660000  message #1257319     Welcome to eclipse.cbi   
1            660000  message #1257319     Welcome to eclipse.cbi   
2            660001  message #1257320  Welcome to eclipse.mbeddr   
3            660001  message #1257320  Welcome to eclipse.mbeddr   
4            660003  message #1257323    Welcome to eclipse.kura   
5            660003  message #1257323    Welcome to eclipse.kura   
6            660004  message #1257324     Welcome to eclipse.cbi   
7            660004  message #1257324     Welcome to eclipse.cbi   
8            660005  message #1257325  Welcome to eclipse.mbeddr   
9            660005  message #1257325  Welcome to eclipse.mbeddr   
10           660006  message #1257326    Welcome to eclipse.kura   
11           660006  message #1257326    Welcome to eclipse.kura   
12           660007  message #1257327     Welcome to eclipse.cbi   
13           660007  message #1257327     Welcome to eclipse.cbi   
14           660008  message #1257328  Welcome to eclipse.mbeddr   
15           660008  message #1257328  Welcome to eclipse.mbeddr   
16           660009  message #1257329    Welcome to eclipse.kura   
17           660009  message #1257329    Welcome to eclipse.kura   
18           660010  message #1257330     Welcome to eclipse.cbi   
19           660010  message #1257330     Welcome to eclipse.cbi   
20           660011  message #1257331  Welcome to eclipse.mbeddr   
21           660011  message #1257331  Welcome to eclipse.mbeddr   
22           660012  message #1257332    Welcome to eclipse.kura   
23           660012  message #1257332    Welcome to eclipse.kura   
24           660013  message #1257333     Welcome to eclipse.cbi   
25           660013  message #1257333     Welcome to eclipse.cbi   
26           660014  message #1257334  Welcome to eclipse.mbeddr   
27           660014  message #1257334  Welcome to eclipse.mbeddr   
28           660015  message #1257335    Welcome to eclipse.kura   
29           660015  message #1257335    Welcome to eclipse.kura   
...             ...               ...                        ...   
369021       879981  message #1497787     Welcome to eclipse.che   
369022       879981  message #1497787     Welcome to eclipse.che   
369023       879982  message #1497788   Welcome to eclipse.titan   
369024       879982  message #1497788   Welcome to eclipse.titan   
369025       879983  message #1497789    Welcome to eclipse.camf   
369026       879983  message #1497789    Welcome to eclipse.camf   
369027       879985  message #1497791   Welcome to eclipse.vorto   
369028       879985  message #1497791   Welcome to eclipse.vorto   
369029       879986  message #1497792     Welcome to eclipse.che   
369030       879986  message #1497792     Welcome to eclipse.che   
369031       879987  message #1497793   Welcome to eclipse.titan   
369032       879987  message #1497793   Welcome to eclipse.titan   
369033       879988  message #1497794    Welcome to eclipse.camf   
369034       879988  message #1497794    Welcome to eclipse.camf   
369035       879990  message #1497796   Welcome to eclipse.vorto   
369036       879990  message #1497796   Welcome to eclipse.vorto   
369037       879991  message #1497797     Welcome to eclipse.che   
369038       879991  message #1497797     Welcome to eclipse.che   
369039       879992  message #1497798   Welcome to eclipse.titan   
369040       879992  message #1497798   Welcome to eclipse.titan   
369041       879993  message #1497799    Welcome to eclipse.camf   
369042       879993  message #1497799    Welcome to eclipse.camf   
369043       879995  message #1497801   Welcome to eclipse.vorto   
369044       879995  message #1497801   Welcome to eclipse.vorto   
369045       879996  message #1497802     Welcome to eclipse.che   
369046       879996  message #1497802     Welcome to eclipse.che   
369047       879997  message #1497803   Welcome to eclipse.titan

# Remove links if the author is Eclipse Webmaster

In [9]:
cols = ['file_number','message_id','topic','author','link','domain','top_level_domain']
new_df_link = df_link[cols][df_link[cols]['author'] != 'Eclipse Webmaster']
new_df_link

file_number        message_id  \
70           660036  message #1257358   
71           660036  message #1257358   
138          660072  message #1257401   
139          660072  message #1257401   
140          660072  message #1257401   
141          660072  message #1257401   
142          660072  message #1257401   
143          660072  message #1257401   
144          660072  message #1257401   
145          660072  message #1257401   
146          660072  message #1257401   
147          660072  message #1257401   
148          660072  message #1257401   
149          660072  message #1257401   
150          660072  message #1257401   
337          660169  message #1257509   
338          660169  message #1257509   
339          660169  message #1257509   
430          660221  message #1257567   
491          660255  message #1257606   
492          660255  message #1257606   
493          660255  message #1257606   
494          660255  message #1257606   
495          660255  message #1257606   
610          660315  message #1257685   
623          660323  message #1257695   
624          660323  message #1257695   
961          660494  message #1257894   
1202         660615  message #1258021   
1521         660777  message #1258189   
...             ...               ...   
366365       878188  message #1495890   
366402       878213  message #1495919   
366403       878213  message #1495919   
366404       878213  message #1495919   
366483       878266  message #1495982   
366778       878473  message #1496197   
366779       878474  message #1496198   
366846       878519  message #1496244   
366847       878519  message #1496244   
367550       878994  message #1496733   
367851       879210  message #1496965   
367966       879289  message #1497058   
368141       879416  message #1497201   
368142       879416  message #1497201   
368143       879416  message #1497201   
368144       879416  message #1497201   
368145       879416  message #1497201   
368200       879455  message #1497241   
368219       879468  message #1497254   
368220       879468  message #1497254   
368221       879468  message #1497254   
368222       879468  message #1497254   
368223       879468  message #1497254   
368224       879468  message #1497254   
368225       879468  message #1497254   
368226       879468  message #1497254   
368227       879468  message #1497254   
368228       879468  message #1497254   
368229       879468  message #1497254   
368484       879639  message #1497432   

                                                    topic  \
70       How to extend jubula for Pshelf widget (nebula)?   
71       How to extend jubula for Pshelf widget (nebula)?   
138                Invoke .emf from Ant programatically ?   
139                Invoke .emf from Ant programatically ?   
140                Invoke .emf from Ant programatically ?   
141                Invoke .emf from Ant programatically ?   
142                Invoke .emf from Ant programatically ?   
143                Invoke .emf from Ant programatically ?   
144                Invoke .emf from Ant programatically ?   
145                Invoke .emf from Ant programatically ?   
146                Invoke .emf from Ant programatically ?   
147                Invoke .emf from Ant programatically ?   
148                Invoke .emf from Ant programatically ?   
149                Invoke .emf from Ant programatically ?   
150                Invoke .emf from Ant programatically ?   
337            Traverse listener not firing on Arrow-Keys   
338            Traverse listener not firing on Arrow-Keys   
339            Traverse listener not firing on Arrow-Keys   
430                                       Sirius Roadshow   
491     Problem building efxclipse sample app with tyc...   
492     Problem building efxclipse sample app with tyc...   
493     Problem building efxclipse sample app with tyc...   
494     Problem building efxclipse sample app with tyc...   
4

In [10]:
new_df_link.to_csv(analyze_dir + 'csv_files/links_list/cleaned/links_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
